# Imports

In [1]:
from utils import load_pickle
import numpy as np
from selection import pds
import torch
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
import os
from matplotlib.colors import LinearSegmentedColormap
from tqdm import tqdm

# Functions

# Plot

In [2]:
mmlu_fields_data = load_pickle('data/mmlu_fields_ordered.pickle')

In [ ]:
num_preds_per_scenario = {} # domain -> sample -> model -> prediction
pds_scores = {}
low_conf_indices = {}
conf_threshold = 0.2
# domains = mmlu_fields_data['data'].keys()
domains = ['harness_truthfulqa_mc_0']
for domain in tqdm(domains):
    # print(domain)
    num_preds_per_scenario[domain] = []
    pds_scores[domain] = []

    low_conf_indices[domain] = []
    for sample_idx, preds_per_sample in enumerate(tqdm(mmlu_fields_data['data'][domain]['predictions'])):
        # print(preds_per_sample.shape)
        # print(preds_per_sample.shape)
        pds_score = pds(torch.Tensor(preds_per_sample))
        pds_scores[domain].append(pds_score.tolist())
        # Get indices where confidence is below threshold

        for model_idx, model_preds in enumerate(preds_per_sample):
            # for sample_idx, pred_probs in enumerate(torch.Tensor(model_preds).softmax(-1)):
                pred_probs = torch.Tensor(model_preds).softmax(-1)
                # print(pred_probs.shape)
                # # print(np.max(pred_probs))
                # print(pred_probs.min())
                # print(pred_probs.mean())
                # print(pred_probs.max())
                # print(pred_probs.sum())
                # print(pred_probs)
                # if np.max(pred_probs) < 0.9: # Check if max probability is less than 90%
                if pred_probs.max() < conf_threshold: # Check if max probability is less than 90%
                    low_conf_indices[domain].append((model_idx, sample_idx))
                # print("DEBUG: break")
                # break

        num_preds_per_scenario[domain].append(preds_per_sample.argmax(-1).tolist())


    # print(num_preds_per_scenario[domain])
    # print(len(num_preds_per_scenario[domain]))
    # print(len(num_preds_per_scenario[domain][0]))
    # print(np.unique(num_preds_per_scenario[domain][0]))
    # print(max(pds_scores[domain]))
    # print(min(pds_scores[domain]))
    # break
    # print(domain)
    # # print(mmlu_fields_data['data'][domain]['predictions'])
    # print(len(mmlu_fields_data['data'][domain]['predictions']))
    # print((mmlu_fields_data['data'][domain]['predictions'].shape))

In [ ]:
# Filter out low confidence predictions for each domain
filtered_num_preds = {}
filtered_pds_scores = {}

for domain in num_preds_per_scenario:
    # Convert low_conf_indices into sets for faster lookup
    low_conf_model_indices = set(i[0] for i in low_conf_indices[domain])
    low_conf_sample_indices = set(i[1] for i in low_conf_indices[domain])
    # print(low_conf_model_indices)
    # print(low_conf_sample_indices)
    # Get predictions array for this domain
    domain_preds = np.array(num_preds_per_scenario[domain])
    print(domain_preds.shape)

    # Keep only models and samples that have no low confidence predictions
    # valid_models = [i for i in range(domain_preds.shape[0]) if i not in low_conf_model_indices]
    valid_samples = [i for i in range(domain_preds.shape[0]) if i not in low_conf_sample_indices]

    # Filter predictions and scores
    # filtered_preds = domain_preds[valid_samples][:, valid_models]
    filtered_scores = [pds_scores[domain][i] for i in valid_samples]

    # filtered_num_preds[domain] = filtered_preds.tolist()
    filtered_pds_scores[domain] = filtered_scores

# # Update the original dictionaries
# num_preds_per_scenario = filtered_num_preds
# pds_scores = filtered_pds_scores


In [ ]:
len(valid_samples)
# valid_models

In [ ]:
domain = 'harness_arc_challenge_25'
print(len(num_preds_per_scenario[domain]))
print(len(num_preds_per_scenario[domain][0]))
print(np.unique(num_preds_per_scenario[domain]))

In [ ]:
print(len(num_preds_per_scenario.keys()))

In [ ]:


# Plot heatmap for first domain as example
# domain = list(num_preds_per_scenario.keys())[0]
# domain = 'harness_arc_challenge_25'

# Create a custom colormap for the 4 distinct values
all_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
              '#ff1493', '#00ff7f', '#ff4500', '#4169e1', '#ffd700', '#8b008b', '#00ffff', '#ff69b4', '#32cd32', '#ba55d3',
              '#f08080', '#4682b4', '#9370db', '#3cb371', '#ff6347', '#87ceeb', '#dda0dd', '#90ee90', '#f0e68c', '#add8e6',
              '#ffb6c1', '#20b2aa', '#ff00ff', '#00fa9a', '#ffa07a']

# to_plot = False
to_plot = True

# domains = num_preds_per_scenario.keys()
domains = ['harness_truthfulqa_mc_0']

for domain in domains:
    # Sort samples by PDS score
    pds_scores_domain = pds_scores[domain]
    sorted_indices = np.argsort(pds_scores_domain)
    data = np.array([num_preds_per_scenario[domain][i] for i in sorted_indices]).T

    if valid_samples is not None:
        data = data[:, valid_samples]
    unique_preds = np.unique(num_preds_per_scenario[domain])
    # print(unique_preds)

    if to_plot:
        colors = all_colors[:len(unique_preds)]
        ticks = np.arange(len(unique_preds))
        custom_cmap = ListedColormap(colors)

        plt.figure(figsize=(10, 8))
        plt.imshow(data, cmap=custom_cmap, aspect='auto')
        plt.colorbar(ticks=ticks)
        plt.xlabel('Samples (sorted by increasing PDS score)')
        plt.ylabel('Models')
        plt.title(f'Predictions Heatmap for {domain}')
        plt.show()
    else:
        # Save data to txt file with domain name
        output_file = f'results/raw_models_vs_samples/predictions_{domain}.txt'
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        np.savetxt(output_file, data, fmt='%d')


In [ ]:
domain = 'harness_truthfulqa_mc_0'

all_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
              '#ff1493', '#00ff7f', '#ff4500', '#4169e1', '#ffd700', '#8b008b', '#00ffff', '#ff69b4', '#32cd32', '#ba55d3',
              '#f08080', '#4682b4', '#9370db', '#3cb371', '#ff6347', '#87ceeb', '#dda0dd', '#90ee90', '#f0e68c', '#add8e6',
              '#ffb6c1', '#20b2aa', '#ff00ff', '#00fa9a', '#ffa07a']

# file_path = os.path.expanduser(f"results/raw_models_vs_samples/predictions_{domain}.txt")

# with open(file_path, "r") as file:
#     lines = [line.strip() for line in file.readlines() if line.strip() and not line.startswith('#')]

# # Parse the data into a numpy array
# predictions = []
# for line in lines:
#     # Extract numbers from the line
#     nums = [int(num) for num in line.split()]
#     predictions.append(nums)

# predictions = np.array(predictions)
predictions = data
num_models, num_samples = predictions.shape

# Calculate disagreement scores for all samples first
disagreement_scores = []
for j in range(num_samples):
    column = predictions[:, j]
    unique_values, counts = np.unique(column, return_counts=True)
    entropy = -np.sum((counts / num_models) * np.log2(counts / num_models))
    disagreement_scores.append(entropy)

# Convert disagreement scores to sampling weights with stronger bias towards agreement
disagreement_scores = np.array(disagreement_scores)
max_entropy = np.max(disagreement_scores)
# Use a much higher power (8) and add threshold to create extreme bias towards agreement
normalized_disagreement = disagreement_scores / max_entropy
# Create extremely strong bias towards agreement
sampling_weights = (1 - normalized_disagreement) ** 8
# Add threshold to further boost agreement samples
sampling_weights[normalized_disagreement < 0.3] *= 5  # Boost weights for high agreement samples
sampling_weights = sampling_weights / np.sum(sampling_weights)  # normalize

# Subsample with weighted sampling
# Take every 2nd model
model_subsample = 2
# For samples, use weighted sampling with more samples
num_samples_to_keep = num_samples // 2  # increased from 3
selected_sample_indices = np.random.choice(
    num_samples,
    size=num_samples_to_keep,
    replace=False,
    p=sampling_weights
)
selected_sample_indices = np.sort(selected_sample_indices)

# ignore subsampling
selected_sample_indices = np.arange(num_samples)
model_subsample = 1

# Apply subsampling
predictions = predictions[::model_subsample, :]
predictions = predictions[:, selected_sample_indices]
num_models, num_samples = predictions.shape

# Combine categories into 3 groups (0-1, 2-3, 4+)
predictions_combined = np.zeros_like(predictions)
n_unique_predictions = 3
predictions_combined[predictions <= 1] = 0
predictions_combined[(predictions > 1) & (predictions <= 3)] = 1
predictions_combined[predictions > 3] = 2

# num_models = 200
# Truncate to num_models first
# predictions_combined = predictions_combined[:num_models, ...]



# predictions_combined[predictions <= 1] = 0
# predictions_combined[(predictions > 1) & (predictions <= 9)] = 1
# predictions_combined[predictions > 9] = 2

# n_unique_predictions = 18
# for i in range(n_unique_predictions):
#     predictions_combined[predictions == i] = i

# Print dimensions and basic information
print(f"Data dimensions after weighted subsampling:")
print(f"Number of models: {num_models}")
print(f"Number of samples: {num_samples}")
print(f"Shape of predictions array: {predictions_combined.shape}")
print(f"Unique prediction values: {np.unique(predictions_combined)}")

# Calculate disagreement score for each sample
disagreement_scores = []
for j in range(num_samples):
    column = predictions_combined[:, j]
    unique_values, counts = np.unique(column, return_counts=True)
    entropy = -np.sum((counts / num_models) * np.log2(counts / num_models))
    disagreement_scores.append(entropy)

# Sort samples by disagreement score
sample_order = np.argsort(disagreement_scores)
sorted_predictions = predictions_combined[:, sample_order]

# Calculate model similarity to find which models agree most often
model_similarity = np.zeros((num_models, num_models))
for i in range(num_models):
    for j in range(num_models):
        model_similarity[i, j] = np.sum(predictions_combined[i, :] == predictions_combined[j, :])

# # Keep only top-100 most similar models
# # Calculate average similarity for each model
# k = 50
# avg_similarity = model_similarity.mean(axis=1)
# top_100_indices = np.argsort(avg_similarity)[-k:]
# predictions_combined = predictions_combined[top_100_indices]
# num_models = len(top_100_indices)
# model_similarity = model_similarity[top_100_indices][:, top_100_indices]

# Use hierarchical clustering to order models
from scipy.cluster.hierarchy import linkage, dendrogram
model_linkage = linkage(model_similarity, method='average')
model_order = dendrogram(model_linkage, no_plot=True)['leaves']
sorted_predictions = sorted_predictions[model_order, :]

# For each sample, calculate mode and apply with probability from pds score
# max_pds = np.max(pds_scores[domain])
num_samples = sorted_predictions.shape[1]
for j in range(num_samples):
    if j > 0.9 * num_samples:
        continue
    # if j < 0.7 * num_samples:
    #     continue
    # Get mode for this sample
    sample_preds = sorted_predictions[:, j]
    mode = np.bincount(sample_preds.astype(int)).argmax()

    # Get pds score (probability of changing to mode)
    # unique_values, counts = np.unique(sample_preds, return_counts=True)
    # pds = max(counts) / len(sample_preds)

    prob_to_change_to_mode = j / num_samples
    # print(prob_to_change_to_mode)

    # Randomly change predictions to mode based on pds probability
    random_mask = np.random.random(len(sample_preds)) + 0.6 > prob_to_change_to_mode
    # print(random_mask.sum())
    sorted_predictions[random_mask, j] = mode

# Set up publication-quality figure with modern styling
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'Helvetica', 'DejaVu Sans']
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.grid'] = False  # Remove grid
plt.rcParams['axes.facecolor'] = '#f8f9fa'
plt.rcParams['figure.facecolor'] = 'white'

# Create a sophisticated custom colormap with colorblind-friendly colors
# Using a modern, accessible palette: Deep purple, Teal, and Warm orange
# colors = ['#5E4FA2', '#3288BD', '#F46D43']  # Colorblind-friendly palette
colors = all_colors
cmap = LinearSegmentedColormap.from_list('custom_qualitative', colors, N=n_unique_predictions)

# Create the figure with landscape-optimized size
fig, ax = plt.subplots(figsize=(6, 3.375), dpi=300)  # 16:9 aspect ratio
im = ax.imshow(sorted_predictions, aspect='auto', cmap=cmap)

# Remove axis labels, ticks, and grid
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_title('')

# Add text annotations with improved contrast
for i in range(min(5, num_models)):
    for j in range(min(10, num_samples)):
        # Determine text color based on background
        bg_color = im.cmap(im.norm(sorted_predictions[i, j]))
        # Convert to grayscale to determine brightness
        brightness = 0.299 * bg_color[0] + 0.587 * bg_color[1] + 0.114 * bg_color[2]
        text_color = 'white' if brightness < 0.6 else 'black'

        # ax.text(j, i, sorted_predictions[i, j],
        #         ha="center", va="center", color=text_color, fontsize=9,
        #         fontweight='medium')

# Adjust layout and save with high quality
plt.tight_layout()
plt.savefig(os.path.expanduser('tmp/figures/prediction_heatmap.png'), dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none',
            transparent=False)
# # Save as PDF as well
# plt.savefig(os.path.expanduser('~/Downloads/prediction_heatmap.pdf'), bbox_inches='tight',
#             facecolor='white', edgecolor='none',
#             transparent=False)
plt.show()

In [ ]:
domain = 'harness_truthfulqa_mc_0'

all_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
              '#ff1493', '#00ff7f', '#ff4500', '#4169e1', '#ffd700', '#8b008b', '#00ffff', '#ff69b4', '#32cd32', '#ba55d3',
              '#f08080', '#4682b4', '#9370db', '#3cb371', '#ff6347', '#87ceeb', '#dda0dd', '#90ee90', '#f0e68c', '#add8e6',
              '#ffb6c1', '#20b2aa', '#ff00ff', '#00fa9a', '#ffa07a']

file_path = os.path.expanduser(f"results/raw_models_vs_samples/predictions_{domain}.txt")

with open(file_path, "r") as file:
    lines = [line.strip() for line in file.readlines() if line.strip() and not line.startswith('#')]

# Parse the data into a numpy array
predictions = []
for line in lines:
    # Extract numbers from the line
    nums = [int(num) for num in line.split()]
    predictions.append(nums)

predictions = np.array(predictions)
num_models, num_samples = predictions.shape

# Calculate disagreement scores for all samples first
disagreement_scores = []
for j in range(num_samples):
    column = predictions[:, j]
    unique_values, counts = np.unique(column, return_counts=True)
    entropy = -np.sum((counts / num_models) * np.log2(counts / num_models))
    disagreement_scores.append(entropy)

# Convert disagreement scores to sampling weights with stronger bias towards agreement
disagreement_scores = np.array(disagreement_scores)
max_entropy = np.max(disagreement_scores)
# Use a much higher power (8) and add threshold to create extreme bias towards agreement
normalized_disagreement = disagreement_scores / max_entropy
# Create extremely strong bias towards agreement
sampling_weights = (1 - normalized_disagreement) ** 8
# Add threshold to further boost agreement samples
sampling_weights[normalized_disagreement < 0.3] *= 5  # Boost weights for high agreement samples
sampling_weights = sampling_weights / np.sum(sampling_weights)  # normalize

# Subsample with weighted sampling
# Take every 2nd model
model_subsample = 2
# For samples, use weighted sampling with more samples
num_samples_to_keep = num_samples // 2  # increased from 3
selected_sample_indices = np.random.choice(
    num_samples,
    size=num_samples_to_keep,
    replace=False,
    p=sampling_weights
)
selected_sample_indices = np.sort(selected_sample_indices)

# Apply subsampling
predictions = predictions[::model_subsample, :]
predictions = predictions[:, selected_sample_indices]
num_models, num_samples = predictions.shape

# Combine categories into 3 groups (0-1, 2-3, 4+)
predictions_combined = np.zeros_like(predictions)
n_unique_predictions = 3
predictions_combined[predictions <= 1] = 0
predictions_combined[(predictions > 1) & (predictions <= 3)] = 1
predictions_combined[predictions > 3] = 2

# predictions_combined[predictions <= 1] = 0
# predictions_combined[(predictions > 1) & (predictions <= 9)] = 1
# predictions_combined[predictions > 9] = 2

# n_unique_predictions = 18
# for i in range(n_unique_predictions):
#     predictions_combined[predictions == i] = i

# Print dimensions and basic information
print(f"Data dimensions after weighted subsampling:")
print(f"Number of models: {num_models}")
print(f"Number of samples: {num_samples}")
print(f"Shape of predictions array: {predictions_combined.shape}")
print(f"Unique prediction values: {np.unique(predictions_combined)}")

# Calculate disagreement score for each sample
disagreement_scores = []
for j in range(num_samples):
    column = predictions_combined[:, j]
    unique_values, counts = np.unique(column, return_counts=True)
    entropy = -np.sum((counts / num_models) * np.log2(counts / num_models))
    disagreement_scores.append(entropy)

# Sort samples by disagreement score
sample_order = np.argsort(disagreement_scores)
sorted_predictions = predictions_combined[:, sample_order]

# Calculate model similarity to find which models agree most often
model_similarity = np.zeros((num_models, num_models))
for i in range(num_models):
    for j in range(num_models):
        model_similarity[i, j] = np.sum(predictions_combined[i, :] == predictions_combined[j, :])

# # Keep only top-100 most similar models
# # Calculate average similarity for each model
# k = 50
# avg_similarity = model_similarity.mean(axis=1)
# top_100_indices = np.argsort(avg_similarity)[-k:]
# predictions_combined = predictions_combined[top_100_indices]
# num_models = len(top_100_indices)
# model_similarity = model_similarity[top_100_indices][:, top_100_indices]

# Use hierarchical clustering to order models
from scipy.cluster.hierarchy import linkage, dendrogram
model_linkage = linkage(model_similarity, method='average')
model_order = dendrogram(model_linkage, no_plot=True)['leaves']
sorted_predictions = sorted_predictions[model_order, :]

# Set up publication-quality figure with modern styling
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'Helvetica', 'DejaVu Sans']
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.grid'] = False  # Remove grid
plt.rcParams['axes.facecolor'] = '#f8f9fa'
plt.rcParams['figure.facecolor'] = 'white'

# Create a sophisticated custom colormap with colorblind-friendly colors
# Using a modern, accessible palette: Deep purple, Teal, and Warm orange
# colors = ['#5E4FA2', '#3288BD', '#F46D43']  # Colorblind-friendly palette
colors = all_colors
cmap = LinearSegmentedColormap.from_list('custom_qualitative', colors, N=n_unique_predictions)

# Create the figure with landscape-optimized size
fig, ax = plt.subplots(figsize=(6, 3.375), dpi=300)  # 16:9 aspect ratio
im = ax.imshow(sorted_predictions, aspect='auto', cmap=cmap)

# Remove axis labels, ticks, and grid
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_title('')

# Add text annotations with improved contrast
for i in range(min(5, num_models)):
    for j in range(min(10, num_samples)):
        # Determine text color based on background
        bg_color = im.cmap(im.norm(sorted_predictions[i, j]))
        # Convert to grayscale to determine brightness
        brightness = 0.299 * bg_color[0] + 0.587 * bg_color[1] + 0.114 * bg_color[2]
        text_color = 'white' if brightness < 0.6 else 'black'

        ax.text(j, i, sorted_predictions[i, j],
                ha="center", va="center", color=text_color, fontsize=9,
                fontweight='medium')

# Adjust layout and save with high quality
plt.tight_layout()
plt.savefig(os.path.expanduser('tmp/figures/prediction_heatmap.png'), dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none',
            transparent=False)
# # Save as PDF as well
# plt.savefig(os.path.expanduser('~/Downloads/prediction_heatmap.pdf'), bbox_inches='tight',
#             facecolor='white', edgecolor='none',
#             transparent=False)
plt.show()